1. INSTAL LIBARY
    
    Mengimpor pustaka Pandas untuk memanipulasi data tabular (DataFrame), NumPy untuk operasi numerik, dan io untuk mengolah data yang diunggah bold text

    Metode yang Digunakan: Library Initialization

In [13]:
import pandas as pd
import numpy as np
import io

2. IMPORT FILE

    Mengimpor modul 'files' dari google.colab,

In [14]:
#Digunakan untuk memungkinkan fitur unggah dan unduh file di lingkungan Google Colab
from google.colab import files

3. Pemuatan Data dan Validasi Berkas
    
   Tahap ini berfokus pada perolehan data dari sumbernya. Kami memanggil fungsi unggah agar pengguna dapat memasukkan file yang sudah diterjemahkan ke dalam sesi Colab. Setelah file berhasil dimuat, data diurai dan ditempatkan ke dalam struktur DataFrame Pandas, menjadikannya siap untuk proses cleansing lebih lanjut. Proses ini mencerminkan validasi awal bahwa data berhasil dialihkan.

In [15]:
# Memunculkan tombol untuk memilih dan mengunggah file.
uploaded = files.upload()

# Mendeteksi nama file yang diunggah.
file_name = list(uploaded.keys())[0]

# Memuat data dari file CSV ke DataFrame Pandas (df).
try:
    df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8')))
    print(f"File '{file_name}' berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

Saving Nike_Penjualan_Dibersihkan_Final (2).csv to Nike_Penjualan_Dibersihkan_Final (2).csv
File 'Nike_Penjualan_Dibersihkan_Final (2).csv' berhasil dimuat.


4. Penanganan Data yang Hilang (Missing Values) dan Imputasi

    Blok ini adalah tahapan inti pembersihan data. Kami menerapkan serangkaian keputusan metodologis untuk mengatasi data yang hilang (NaN)

In [16]:
# Membuat salinan data untuk proses pembersihan.
df_clean = df.copy()

# A. Hapus Baris Total Kosong:
# Menghapus baris yang tidak memiliki ID Pesanan.
df_clean.dropna(subset=['ID_Pesanan'], inplace=True)

# B. Imputasi Numerik (Mengisi NaN):
# Mengisi NaN pada Diskon_Diterapkan dan Unit_Terjual dengan 0.
df_clean['Diskon_Diterapkan'] = df_clean['Diskon_Diterapkan'].fillna(0)
df_clean['Unit_Terjual'] = df_clean['Unit_Terjual'].fillna(0).astype(int)

# Mengisi NaN pada MRP dengan nilai median per Lini Produk.
df_clean['Harga_Eceran_Maksimum'] = df_clean.groupby('Lini_Produk')['Harga_Eceran_Maksimum'].transform(lambda x: x.fillna(x.median()))

# C. Imputasi Kategorikal:
# Mengisi nilai NaN pada kolom Ukuran dengan Modus.
df_clean['Ukuran'] = df_clean['Ukuran'].fillna(df_clean['Ukuran'].mode()[0])

5. Koreksi Tipe Data dan Penanganan Inkonsistensi Logis


    Bagian ini berfokus pada validasi dan standardisasi struktural data. Kami mengonversi kolom ID menjadi tipe bilangan bulat (integer) dan memperbaiki format Tanggal_Pesanan menjadi tipe datetime untuk memungkinkan analisis temporal. Lebih lanjut, kami menghapus anomali fatal di mana kerugian dicatat tanpa adanya pendapatan yang jelas, karena kondisi ini secara logis mustahil dan mengindikasikan error pencatatan yang parah

    Metode yang digunakan: Type Casting dan Anomaly Handling (Penghapusan Logis)
  

In [17]:
# A. Koreksi Tipe Data
# Mengonversi ID Pesanan menjadi integer.
df_clean['ID_Pesanan'] = df_clean['ID_Pesanan'].astype(int)

# Mengonversi Tanggal Pesanan ke format datetime.
df_clean['Tanggal_Pesanan'] = pd.to_datetime(df_clean['Tanggal_Pesanan'], errors='coerce')

# B. Penanganan Inkonsistensi Logis:
# Menghapus baris di mana Pendapatan = 0 DAN Keuntungan < 0.
df_clean = df_clean[~((df_clean['Pendapatan'] == 0) & (df_clean['Keuntungan'] < 0))]

6.  Standardisasi Kolom Ukuran

    Bagian ini fokus pada standardisasi data kategorikal. Nilai numerik yang tidak baku pada kolom Ukuran dikelompokkan ke dalam kategori standar (S, M, L, XL) untuk memastikan konsistensi analisis

In [19]:
# --- STANDARDISASI UKURAN ---

# Membuat peta pemetaan dari ukuran non-standar/numerik ke standar (S, M, L, XL)
size_mapping = {
    # Numerik ke Kategori
    '6': 'S', '7': 'S',
    '8': 'M', '9': 'M',
    '10': 'L', '11': 'L',
    '12': 'XL',
    # Kategori sudah ada (untuk memastikan)
    'M': 'M', 'L': 'L', 'XL': 'XL'
}

# Mengonversi kolom Ukuran ke string untuk menangani nilai numerik dan teks secara seragam
df_clean['Ukuran'] = df_clean['Ukuran'].astype(str)

# Menerapkan pemetaan standardisasi ukuran
df_clean['Ukuran'] = df_clean['Ukuran'].replace(size_mapping)

print("\nStandardisasi kolom 'Ukuran' berhasil diterapkan.")
print("Nilai unik akhir di kolom Ukuran:", df_clean['Ukuran'].unique())


Standardisasi kolom 'Ukuran' berhasil diterapkan.
Nilai unik akhir di kolom Ukuran: ['M' 'L' 'XL' 'S']


7.  Ringkasan Akhir dan Penyimpanan Hasi

    Sebagai tahap finalisasi dan penutupan, blok ini memberikan ringkasan status data (perbandingan jumlah baris awal dan akhir) dan menyajikan data yang telah dibersihkan. Selanjutnya, data yang sudah tervalidasi tersebut disimpan ke dalam file CSV baru dan secara otomatis diunduh ke perangkat pengguna.

    Metode yang Digunakan: Data Serialization (Serialisasi Data) dan Validation

In [22]:
# Ringkasan Akhir dan Penyimpanan
print(f"Bentuk Awal Data: {df.shape[0]} baris")
print(f"Bentuk Akhir Data Bersih: {df_clean.shape[0]} baris")

print("\n5 Baris Pertama Data yang Sudah Dibersihkan:")
print(df_clean.head().to_markdown(index=False))

# Menyimpan DataFrame yang sudah dibersihkan ke file CSV baru.
output_file_name_clean = 'Nike_Penjualan_Dibersihkan_Final.csv'
df_clean.to_csv(output_file_name_clean, index=False)

# Mengunduh file hasil (fitur Colab).
files.download(output_file_name_clean)
print(f"\nDataset akhir telah disimpan sebagai '{output_file_name_clean}' dan diunduh ke komputer Anda.")

Bentuk Awal Data: 587 baris
Bentuk Akhir Data Bersih: 587 baris

5 Baris Pertama Data yang Sudah Dibersihkan:
|   ID_Pesanan | Kategori_Gender   | Lini_Produk   | Nama_Produk    | Ukuran   |   Unit_Terjual |   Harga_Eceran_Maksimum |   Diskon_Diterapkan |   Pendapatan | Tanggal_Pesanan     | Saluran_Penjualan   | Wilayah   |   Keuntungan |
|-------------:|:------------------|:--------------|:---------------|:---------|---------------:|------------------------:|--------------------:|-------------:|:--------------------|:--------------------|:----------|-------------:|
|         2002 | Wanita            | Sepak Bola    | Premier III    | M        |              4 |                 5937.52 |                   0 |            0 | NaT                 | Ritel               | Mumbai    |      3337.34 |
|         2003 | Anak              | Gaya Hidup    | Blazer Mid     | L        |              0 |                 9673.57 |                   0 |            0 | 2024-04-10 00:00:00 | Daring     

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Dataset akhir telah disimpan sebagai 'Nike_Penjualan_Dibersihkan_Final.csv' dan diunduh ke komputer Anda.
